# Bwamin VS BWA Benchmark

Note: Bwamin --mem --sw will fail for Large + files. DO NOT RUN!

## Table of Contents
Note: Click to jump to section
1. [Installation Check](#installation)
2. [Time Section](#time)
    1. BWT
    2. SW
    3. BWA
    4. Graphs
3. [Memory Section](#memory)
    1. BWT
    2. SW
    3. BWA
    4. Graphs
4. [Conclusions](#conclusions)

## Installation Check <a name="installation"></a>

In [3]:
%%capture
!echo hi
# Run this cell to make capture small again

Let's check if BWA and Bwamin are installed

In [4]:
!bwa


Program: bwa (alignment via Burrows-Wheeler transformation)
Version: 0.7.17-r1188
Contact: Heng Li <lh3@sanger.ac.uk>

Usage:   bwa <command> [options]

Command: index         index sequences in the FASTA format
         mem           BWA-MEM algorithm
         fastmap       identify super-maximal exact matches
         pemerge       merge overlapping paired ends (EXPERIMENTAL)
         aln           gapped/ungapped alignment
         samse         generate alignment (single ended)
         sampe         generate alignment (paired ended)
         bwasw         BWA-SW for long queries

         shm           manage indices in shared memory
         fa2pac        convert FASTA to PAC format
         pac2bwt       generate BWT from PAC
         pac2bwtgen    alternative algorithm for generating BWT
         bwtupdate     update .bwt to the new format
         bwt2sa        generate SA from BWT and Occ

Note: To use BWA, you need to first index the genome with `bwa index'.
      There are

In [8]:
!python bwamin.py -h

usage: bwamin.py [-h] [--index] [--mem] [--bwt] [--sw] [--fasta FASTA]
                 [--fastq FASTQ] [-A A] [-B B] [-O O] [-E E] [-out OUT]

minimum bwa

options:
  -h, --help            show this help message and exit
  --index               index a fasta file
  --mem                 index a fasta file
  --bwt                 enables Burrow-Wheeler Transform search
  --sw                  enables Smith-Waterman search
  --fasta FASTA, --fa FASTA
                        fasta input
  --fastq FASTQ, --fq FASTQ
                        fastq input
  -A A                  Matching score. [1]
  -B B                  Mismatch penalty. The sequence error rate is
                        approximately: {.75 * exp[-log(4) * B/A]}. [4]
  -O O                  Gap open penalty. [6]
  -E E                  Gap extension penalty. A gap of length k costs O + k*E
                        (i.e. -O is for opening a zero-length gap). [1]
  -out OUT              Specify output file


## Time Section <a name="time"></a>

We will test bwamin bwt, bwamin sw, and bwa on a custom
short, and longer files.

In [41]:
# Helper commands so we can graph later
import pandas as pd
import seaborn as sns

# Change string to seconds
def timing(string):
    if type(string) == type(0.01):
        return string
    elements = string.split('m')
    minutes = elements[0]
    seconds = elements[1]

    seconds = seconds.replace('s', '')
    minues = minutes.strip()
    seconds = seconds.strip()

    if not minutes.isnumeric() or seconds.isnumeric():
        print('ERROR')

    return (float(minutes) * 60 + float(seconds))
    
def convertTime(filename):
    if '.txt' not in filename:
        filename = filename + '.txt'
    x = pd.read_csv(filename, sep='\t', header=None)
    x[1] = x[1].apply(timing)
    x.to_csv(filename, header=False, index=False, sep='\t')     
    
# Summing the time with index and mem to one file
def addTime(filename):
    # Read files
    x = pd.read_csv(filename + "Index.txt", sep='\t', header=None)
    y = pd.read_csv(filename + "Mem.txt", sep='\t', header=None)
    
    # Convert and add times
    x[1] = x[1].apply(timing)
    y[1] = y[1].apply(timing)
    x[1] = x[1] + y[1]
    
    # Write to file
    x.to_csv(filename + '.txt', header=False, index=False, sep='\t') 
    
def convertSec(filename):
    if '.txt' not in filename:
        filename = filename + '.txt'
    x = pd.read_csv(filename, sep='\t', header=None)
    x[1] = x[1].apply(timing)
    return x[1][0]
    
def addSec(filename):
    # Read files
    x = pd.read_csv(filename + "Index.txt", sep='\t', header=None)
    y = pd.read_csv(filename + "Mem.txt", sep='\t', header=None)
    # Convert and add times
    x[1] = x[1].apply(timing)
    y[1] = y[1].apply(timing)
    x[1] = x[1] + y[1]
    return x[1][0]

In [42]:
# Initiallize the values total time
bwt = [0 for i in range(5)]
sw = [0 for j in range(5)]
bwa = [0 for k in range(5)]


0.632

### Short

In [12]:
# Peak at short reference and read
print('short.fa')
!head benchmark/testfiles/short.fa
print('\n\nshort.fq')
!head benchmark/testfiles/short.fq

short.fa
>chr1
TGCAA

short.fq
@Sample1
TCAT
+
!1=D

In [44]:
%%capture
# Bwt
!{ time python bwamin.py --index --bwt --fa benchmark/testfiles/short.fa \
  2> sleep.stderr ;} 2> benchmark/testfiles/bwaminBwtShortTimeIndex.txt
!{ time python bwamin.py --mem --bwt --fa benchmark/testfiles/short.fa \
  --fq benchmark/testfiles/short.fq -out benchmark/testfiles/bwtshort.sam \
  2> sleep.stderr ;} 2> benchmark/testfiles/bwaminBwtShortTimeMem.txt
bwt[0] = addSec('benchmark/testfiles/bwaminBwtShortTime')

# Sw
!{ time python bwamin.py --mem --sw --fa benchmark/testfiles/short.fa\
  --fq benchmark/testfiles/short.fq -out benchmark/testfiles/swshort.sam\
  2> sleep.stderr ;} 2> benchmark/testfiles/bwaminSwShortTime.txt
sw[0] = convertSec('benchmark/testfiles/bwaminSwShortTime.txt')

# Bwa
!{ time bwa index benchmark/testfiles/short.fa \
  2> sleep.stderr ;} 2> benchmark/testfiles/bwaShortTimeIndex.txt
!{ time bwa mem benchmark/testfiles/short.fa \
  benchmark/testfiles/short.fq > benchmark/testfiles/bwashort.sam\
  2> sleep.stderr ;} 2> benchmark/testfiles/bwaShortTimeMem.txt
bwa[0] = addSec('benchmark/testfiles/bwaShortTime')

In [46]:
# Print Short Results
print('Bwt')
!cat benchmark/testfiles/bwaminBwtShortTime.txt
print('\nSw')
!cat benchmark/testfiles/bwaminSwShortTime.txt
print('\nBwa')
!cat benchmark/testfiles/bwaShortTime.txt

Bwt
real	1.721
user	1.116
sys	0.362

Sw

real	0m0.724s
user	0m0.544s
sys	0m0.169s

Bwa
real	0.033
user	0.006
sys	0.013


## Medium

In [15]:
# Peak at medium reference and read
print('testfastring.fa')
!head benchmark/testfiles/testfastring.fa
print('\n\ntestfqstring.fq')
!head benchmark/testfiles/testfqstring.fq

testfastring.fa
>chr1
ATCCTATATTACGACTTTGGCAGGGGGTTCGCAAGTCCCACCCCAAACGATGCTGAAGGCTCAGGTTACACAGGCACAAGTACTATATATACGAGTTCCCGCTCTTAACCTGGATCGAATGCAGAATCATGCATCGTACCACTGTGTTCGTGTCATCTAGGACGGGCGCAAAGGATATATAATTCAATTAAGAATACCTTATATTATTGTACACCTACCGGTCACCAGCCAACAATGTGCGGATGGCGTT
>chr2
ACGACTTACTGGGCCTGATCTCACCGCTTTAGATACCGCACACTGGGCAATACGAGGTAAAGCCAGTCACCCAGTGTCGATCAACAGCTAACGTAACGGTAAGAGGCTCACAAAATCGCACTGTCGGCGTCCCTTGGGTATTTTACGTTAGCATCAGGTGGACTAGCATGAATCTTTACTCCCAGGCGAAAACGGGTGCGTGGACAAGCGAGCAGCAAACGAAAATTCTTGGCCTGCTTGGTGTCTCGTA

testfqstring.fq
@SRR5077691.13
NTGAAAAGATGTCTCCTTCTGTAAGTCAGAACAAAAAACTTTAATTAACT
+
!1=DDFFFHHHHGJJJJJJJJIJJJIIIJIHGIJJJJJJJJJFHHIJIJI
@SRR5077691.79
GTTTCATTGTGTCTTTATTTCCTGTATTAATGAGATGGGATATGAAGTCT
+
JJJJJJJIJJIJJJJJJJJIHJJJJJJJJJJJJJJJJHHHHHFFFFF@CB
@SRR5077691.32
GCCGTGTGCCCCCTCTTGGGTGACACCCCACCCCACCCTTATTTGCATCN


In [47]:
%%capture
# Bwt
!{ time python bwamin.py --index --bwt --fa benchmark/testfiles/testfastring.fa 2> sleep.stderr ;} 2> benchmark/testfiles/bwaminBwtMedTimeIndex.txt
!{ time python bwamin.py --mem --bwt --fa benchmark/testfiles/testfastring.fa --fq benchmark/testfiles/testfqstring.fq -out benchmark/testfiles/bwttest.sam 2> sleep.stderr ;} 2> benchmark/testfiles/bwaminBwtMedTimeMem.txt
bwt[1] = addSec('benchmark/testfiles/bwaminBwtMedTime')

# Sw
!{ time python bwamin.py --mem --sw\
  --fa benchmark/testfiles/testfastring.fa\
  --fq benchmark/testfiles/testfqstring.fq \
  -out benchmark/testfiles/swtest.sam \
  2> sleep.stderr ;} 2> benchmark/testfiles/bwaminSwMedTime.txt
sw[1] = convertSec('benchmark/testfiles/bwaminSwMedTime.txt')

# Bwa
!{ time bwa index benchmark/testfiles/testfastring.fa 2> sleep.stderr ;} 2> benchmark/testfiles/bwaMedTimeIndex.txt
!{ time bwa mem benchmark/testfiles/testfastring.fa\
  benchmark/testfiles/testfqstring.fq > benchmark/testfiles/bwatest.sam 2> sleep.stderr ;} 2> benchmark/testfiles/bwaMedTimeMem.txt
bwa[1] = addSec('benchmark/testfiles/bwaMedTime')

In [48]:
# Print Medium Results
print('Bwt')
!cat benchmark/testfiles/bwaminBwtMedTime.txt
print('\nSw')
!cat benchmark/testfiles/bwaminSwMedTime.txt
print('\nBwa')
!cat benchmark/testfiles/bwaMedTime.txt

Bwt
real	1.3860000000000001
user	1.02
sys	0.295

Sw

real	0m1.132s
user	0m0.975s
sys	0m0.143s

Bwa
real	0.043
user	0.007
sys	0.014


### Long

This is an Ebola Virus found using sra.

In [52]:
# Peak at long reference and read
print('SRR10769501.fasta.fixed')
!head -n 4 benchmark/mydata/SRR10769501.fasta.fixed
print('\n\nSRR10769501.fastq')
!head -n 4 benchmark/mydata/SRR10769501.fastq

SRR10769501.fasta.fixed
>@SRR10769501.1.1 M02486:32:000000000-BTHFB:1:1101:16201:1720 length=160
GCCGTAGCCCTGCTCGCCAGCGCGTAGCGGTGTCGTTTCCGTAGCGTCATCTTCGTCATCATTATTTCCAGTGGGTTCCTCGTTTTCACTCGCATTCGTGTCTTCGTCTTCCACCTTGCGAACAAAGTCTTTCTTCCCCCGGATCGCAAAGAGCTCCAGC
>@SRR10769501.2.1 M02486:32:000000000-BTHFB:1:1101:21365:1807 length=160
GGGCTTCACGGGCTTGCGGCGTTTCCACGCCGTGGTCAACGGCGTTGCGCAGCAGGTGGCCCAGCGGGGCATCGAGCATCTGTCTCTTATACACATCTCCGACCCCACTCTACAGGCACAAATCTCCTATTCCGTCTTCTTCTTTAAAACAAAAACCCCC


SRR10769501.fastq
@SRR10769501.1.1 M02486:32:000000000-BTHFB:1:1101:16201:1720 length=160
GCCGTAGCCCTGCTCGCCAGCGCGTAGCGGTGTCGTTTCCGTAGCGTCATCTTCGTCATCATTATTTCCAGTGGGTTCCTCGTTTTCACTCGCATTCGTGTCTTCGTCTTCCACCTTGCGAACAAAGTCTTTCTTCCCCCGGATCGCAAAGAGCTCCAGC
+SRR10769501.1.1 M02486:32:000000000-BTHFB:1:1101:16201:1720 length=160
????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????


In [30]:
%%capture
# Bwt
# Note file too big will not run
# !{ time python bwamin.py --index --bwt\
#   --fa benchmark/mydata/SRR10769501.fasta.fixed \
#   2> sleep.stderr ;} \
#   2> benchmark/mydata/bwaminBwtLongTimeIndex.txt
# !{ time python bwamin.py --mem --bwt\
#   --fa benchmark/mydata/SRR10769501.fasta.fixed\
#   --fq benchmark/mydata/SRR10769501.fastq \
#   2> sleep.stderr ;} \
#   2> benchmark/mydata/bwaminBwtLongTimeMem.txt
# addTime('benchmark/mydata/bwaminBwtLongTime')



In [ ]:
%%capture
# NOTE: This process will be killed even with 48GB, Don't run
# Sw
# !{ time python bwamin.py --mem --sw\
#   --fa benchmark/mydata/SRR10769501.fasta.fixed\
#   --fq benchmark/mydata/SRR10769501.fastq -out swlong.sam\
#   2> sleep.stderr ;} 2> benchmark/mydata/bwaminSwLongTime.txt
# convertTime('benchmark/mydata/bwaminSwLongTime.txt')

In [50]:
%%capture
# Bwa
!{ time bwa index benchmark/mydata/SRR10769501.fasta.fixed \
  2> sleep.stderr ;} 2> benchmark/mydata/bwaLongTimeIndex.txt
!{ time bwa mem benchmark/mydata/SRR10769501.fasta.fixed\
  benchmark/mydata/SRR10769501.fastq > benchmark/mydata/bwalong.sam\
  2> sleep.stderr ;} 2> benchmark/mydata/bwaLongTimeMem.txt
bwa[2] = addSec('benchmark/mydata/bwaLongTime')

In [28]:
#Print Short Results
print('Bwt')
!cat benchmark/mydata/bwaminBwtLongTime.txt
print('\nSw')
!cat benchmark/mydata/bwaminSwLongTime.txt
print('\nBwa')
!cat benchmark/mydata/bwaLongTime.txt

Bwt
cat: benchmark/mydata/bwaminBwtLongTime.txt: No such file or directory

Sw
cat: benchmark/mydata/bwaminSwLongTime.txt: No such file or directory

Bwa
real	12.486
user	11.892
sys	0.307


## Ebola Virus 

I found a smaller one with only 19kbases

In [34]:
# SRR10083456.fastq
#bwt
!{ time python bwamin.py --index --bwt\
  --fa benchmark/mydata/SRR10083456.fasta \
  2> sleep.stderr ;} \
  2> benchmark/mydata/bwaminBwtEbolaTimeIndex.txt

!{ time python bwamin.py --mem --bwt\
  --fa benchmark/mydata/SRR10083456.fastq\
  --fq benchmark/mydata/SRR10083456.fastq \
  -out benchmark/mydata/bwtebola.sam 2> sleep.stderr ;} \
  2> benchmark/mydata/bwaminBwtEbolaTimeMem.txt
bwt[3] = addSec('benchmark/mydata/bwaminBwtEbolaTime')

In [ ]:
# Sw
!{ time python bwamin.py --mem --sw\
  --fa benchmark/mydata/SRR10083456.fasta\
  --fq benchmark/mydata/SRR10083456.fastq -out swebola.sam\
  2> sleep.stderr ;} 2> benchmark/mydata/bwaminSwEbolaTime.txt
sw[3] = convertSec('benchmark/mydata/bwaminSwEbolaTime.txt')

In [ ]:
%%capture
# Bwa
!{ time bwa index benchmark/mydata/SRR10083456.fasta \
  2> sleep.stderr ;} 2> benchmark/mydata/bwaEbolaTimeIndex.txt

!{ time bwa mem benchmark/mydata/SRR10083456.fasta\
  benchmark/mydata/SRR10083456.fastq > benchmark/mydata/bwaebola.sam\
  2> sleep.stderr ;} 2> benchmark/mydata/bwaEbolaTimeMem.txt
bwa[3] = addSec('benchmark/mydata/bwaEbolaTime')

In [ ]:
#Print Ebola Results
print('Bwt')
!cat benchmark/mydata/bwaminBwtEbolaTime.txt
print('\nSw')
!cat benchmark/mydata/bwaminSwEbolaTime.txt
print('\nBwa')
!cat benchmark/mydata/bwaEbolaTime.txt

### In-Class sample
We have chromsome 17 from a mouse as the reference and the reads come from Sox2. This was found in lab 5.

Note: Just like in the Long section, BWT and SW will not run to completion due to low memory. They both get killed in the process.

### Graphs

## Artificial Genome Testing

## Conclusion
